In [1]:
import pandas as pd
import os
directory_path = 'prague_final\QnAs'

file_list = [os.path.join(directory_path, filename) for filename in os.listdir(directory_path) if filename.endswith('.txt')]
print(len(file_list))


47


In [2]:
def file_to_df(filenames):

    # Initialize empty lists for questions and answers
    questions = []
    answers = []

    # Loop through each file and extract questions and answers
    for filename in filenames:
        with open(filename, 'r', encoding='utf-8') as f:
            data = [line.strip() for line in f]

        # Loop through each line and append to respective lists
        for i, line in enumerate(data):
            if line.startswith('Q:'):
                questions.append(line[2:])
            elif line.startswith('A:'):
                # Keep appending lines until the next question is encountered
                answer = line[2:]
                j = i + 1
                while j < len(data) and not data[j].startswith('Q:'):
                    answer += ' ' + data[j]
                    j += 1
                answers.append(answer)

    # Create pandas dataframe
    print(len(questions))
    print(len(answers))
    df = pd.DataFrame({'question': questions, 'answer': answers})
    # Add 'id' column as the first column
    # df.insert(0, 'id', range(1, len(df) + 1))

    # Print dataframe
    print(f"Objects in DataFrame: {len(df.question)}")
    return df


In [5]:
df.to_csv('prague_data.csv')

In [3]:
file_list[40]

'prague_final\\QnAs\\QnA_data_45.txt'

In [4]:
df = file_to_df(file_list)

790
790
Objects in DataFrame: 790


In [1]:
import weaviate
import cohere
import json
auth_config = weaviate.auth.AuthApiKey(api_key="averYngrGCxpB0baHuO2klfJL22bG2GOsuNh")
client = weaviate.Client(
    url = "https://rodssvhcqlsp0zfor5qqa.gcp.weaviate.cloud",  # Replace with your endpoint
    auth_client_secret=auth_config,
    additional_headers = {
        "X-Cohere-Api-Key" : "DpxdW7Etapsz2S14AEp73zP0er8H9DCZ9IimztR9" # Replace with your API key
    }  
)

In [ ]:

client.schema.get()
json_data = df.to_json(orient='records')
class_obj = {
    "class": "prague",
    "vectorizer": "text2vec-cohere"  # Or "text2vec-cohere" or "text2vec-huggingface"
}

client.schema.create_class(class_obj)
data = json.loads(json_data)
with client.batch as batch:
    batch.batch_size=100
    for i, d in enumerate(data):
        properties = {
            "answer": d["answer"],
            "question": d["question"]
        }

        client.batch.add_data_object(properties, "prague")

In [2]:
client.schema.get()

{'classes': [{'class': 'Question',
   'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
    'cleanupIntervalSeconds': 60,
    'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
   'moduleConfig': {'text2vec-cohere': {'vectorizeClassName': True}},
   'properties': [{'dataType': ['text'],
     'description': "This property was generated by Weaviate's auto-schema feature on Thu Apr 13 06:10:34 2023",
     'moduleConfig': {'text2vec-cohere': {'skip': False,
       'vectorizePropertyName': False}},
     'name': 'answer',
     'tokenization': 'word'},
    {'dataType': ['text'],
     'description': "This property was generated by Weaviate's auto-schema feature on Thu Apr 13 06:10:34 2023",
     'moduleConfig': {'text2vec-cohere': {'skip': False,
       'vectorizePropertyName': False}},
     'name': 'question',
     'tokenization': 'word'}],
   'replicationConfig': {'factor': 1},
   'shardingConfig': {'virtualPerPhysical': 128,
    'desiredCount': 1,
    'actualCount': 